# 1. Install, Imports, Settings

In [29]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import spacy
from spacy.lang.es.examples import sentences 
#!python -m spacy download es_core_news_md
nlp = spacy.load("es_core_news_md")

import string, math

In [30]:
plt.rcParams["figure.figsize"] = [10, 6]
%config InlineBackend.figure_format = 'retina'

punct = string.punctuation +'”“'

# 2. Loading data

In [31]:
df = pd.read_csv("data/alertas.csv", sep="|")

print("Total Number of documents:", len(df))
print("Number of documents with no accesible text (password protected):", len(df[df['Text'].isnull()]))
print("Number of documents with accesible text:",  len(df[df['Text'].notnull()]))

Total Number of documents: 1753
Number of documents with no accesible text (password protected): 5
Number of documents with accesible text: 1748


In [32]:
df.head()

,Filename,Text,Subtype,Type,Year,Path,Departamento
0,"AT N° 003-18 NAR-Cumbitara, Maguí Payán, Polic...","Defensoria del Pueblo COLOMB IA Bogotá D.C., 5...",Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 003-18 NAR-...,Nariño
1,AT N° 004-18 NAR-Tumaco.pdf,Defensoría del Pueblo CO LO Mll.t. Carrera 9 #...,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 004-18 NAR-...,Nariño
2,AT N° 005-18 COR-Tierralta.pdf,Carrera 9 # 16 -21 Bogotá D.C. PBX: (57) (1) 3...,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 005-18 COR-...,Córdoba
3,AT N° 006-18 ARA-Saravena.pdf,Defensoría del Pueblo Carrera 9 # 16-21 Bogotá...,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 006-18 ARA-...,Arauca
4,"AT N° 007-18 MET-Puerto Lleras, Puerto Rico y ...",San Vicente Bajo l' Margen Izquierda del río G...,Alerta Temprana,Advertencia,2018,data\Advertencia_PDF\AT 2018\AT N° 007-18 MET-...,Meta


# 3. Basic Text Cleaning 

In [33]:
df.loc[25]["Text"]

'Carrera 9 # 16 -21 Bogotá D.C. PBX: (57) (1 ) 3147300 Línea Nacional: 01 8000 914814 www.defensoria.gov.co Plantilla Vigente desde: 27/06/2017 Bogotá D.C., 5 de marzo de 2018 Doctor GUILLERMO RIVERA FLÓREZ Ministro del Interior Secretaría Técnica de la Comisión Intersectorial para la Respuesta Rápida a las Alertas Tempranas (CIPRAT) Carrera 8 No. 12 B - 31 Ciudad Referencia: ALERTA TEMPRANA DE INMINENCIA N° 0 27-181 debido a la situación de riesgo de vulneracio nes a los DDHH y al DIH de los habitantes del municipio Tarazá – Antioquia, de todos los barrios del casco urbano y de los siguientes corregimientos y veredas: Barro Blanc o, veredas: Tahamí, Doradas Altas, Doradas Bajas, Neri, Tesorito, Potrero Largo, Purí, San Antonio, Tamaco, Barro Blanco; Corregimiento El Doce: Puquí, centro pob lado El Doce; Corregimiento La Caucana, veredas: Santa Clara, La Esperanza, Pecoralia, Agualin da, La Primavera, Rancho Viejo, Pecora, La Caucana, La Cabaña, Piedra Brava, La Cidra, El Porvenir, Bla

# 4. Extracting Actores del Conflicto

In [34]:
#https://es.wikipedia.org/wiki/C%C3%A1rtel_(organizaci%C3%B3n_il%C3%ADcita)#Colombia
#https://es.wikipedia.org/wiki/Bandas_y_grupos_emergentes_en_Colombia
#https://www.datos.gov.co/Mapas-Nacionales/Departamentos-y-municipios-de-Colombia/xdk5-pm3f

In [35]:
grupos = {
    "Grupo Armado Organizado" : ["GAO"],
    "Grupo Armado Ilegal" : ["GAI"],
    "Grupo Armado Organizado Residual" : ["GAOR", "GAO-R", "grupo armado organizado - residual" ], 
    "Ejército de Liberación Nacional" : ["ELN"],
    "Autodefensas Gaitanistas de Colombia" : ["AGC"],
    "Fuerzas Armadas Revolucionarias" : ["FARC", ],
    "Grupo delincuencial organizado" : ["GDO", ],
    "Ejército del Pueblo" : ["EP"],
    "Jalisco Nueva Generación": [],
    "Bandas Criminales" : ["BACRIM", "Bandas Emergentes"],
    "Autodefensas Unidas de Colombia" : ["AUC", ],
    "Grupos posdesmovilización": [],
    "Ejército de Liberación Popular": ["EPL"],
    "Clan del Golfo" : ["AGC"],
}

children = ["NNAJ", "niños", "niñas", "adolescentes", "jóvenes", "NNA"]

narco_grupos  = ["Cartel del Amazonas",  
        "Cartel de Los Nevados",
        "Cartel de Bogotá",
        "Cartel de Buga",
        "Cartel de Medellín",
        "Oficina de Envigado",
        "Cartel de Cali",
        "Cartel del Caquetá",
        "Cartel del Norte del Valle",
        "Los Rastrojos",
        "Cartel de la Costa Atlántica",
        "Cartel de La Guajira",
        "Cartel del Cauca",
        "Los Caparrapos",
        "Los Paisas",
        "Águilas Negras",
        "Los Puntilleros",
        "Ejército Revolucionario Popular Antisubversivo de Colombia",
        "Los Pelusos",
        "GDO Los Pachenca",
        "Clan Gnecco Cerchar",
        "Clan Bustamante", 
        "Activo Primer Frente",
        "Segunda Marquetalia", 
        "Los Zetas", 
        "Cartel de Sinaloa", 
        "Oficina de Envigado", 
        "La Terraza",
        "Robledo",
        "Pachelly",
        "La Sierra", 
        "Caicedo", 
        "Los Triana",
        "Clan del Oriente", 
        "La Unión",
        "Clan del Norte",
        "Los Costeños", 
        "Los Pachencha", 
        "La Cordillera",
        "La Gran Alianza",
        "Banda Local",
        "Norte del Valle",
        "Los Chuckys",
        "Los Chacales",
        "La Constru",
        "Los Caqueteños"]

POI = {
    "Evaristo Porras": "Papa Doc",
    "Luis Agustín Caicedo Velandia" : "Don Lucho", 
    "Víctor Manuel Mejía Múnera": "Pablo Arauca",
    "Miguel Ángel Mejía Múnera": "El Mellizo",
    "Ramón Quintero":"",
    "Pablo Escobar" : "El patrón",
    "Gonzalo Rodríguez Gacha": "",
    "Diego Fernando Murillo": "",
    "Gilberto Rodríguez Orejuela": "El Ajedrecista",
    "Miguel Rodríguez Orejuela": "",
    "Leonidas Vargas": "El Rey de Caquetá",
    "Luis Enrique Calle Serna": "",
    "Wilber Varela": "",
    "Alberto Orlandez Gamboa": "", 
    "Hermágoras González": "",
    "Nicolás Rodríguez Bautista" : " Gabino ", 
    "Iván Márquez": "",
    "Gentil Duarte": "",
    "Jorge Gnecco Cerchar": "", 
    "Jesús María Aguirre" :"Chucho Mercancía",
    "Megateo": "",  
    "Vicente Castaño": "", 
    "Emiliano Alcides Osorio": " Caín ",
    "Edgar Bustamante" : "El Yuca",
    "Manuel Marulanda Vélez" :" Tirofijo ",     
    "Jorge Briceño Suarez Mono": " Jojoy ",
    "Dairo Antonio Úsuga David": "",
    "Salvatore Mancuso": "",
    "Rodrigo Tovar": "",
    "Carlos Castaño Gil": "",
    "Justo Pastor Perafán": "Don Pepe"
}

loc = pd.read_csv("resources/Departamentos_y_municipios_de_Colombia.csv")
loc.head()

,REGION,CÓDIGO DANE DEL DEPARTAMENTO,DEPARTAMENTO,CÓDIGO DANE DEL MUNICIPIO,MUNICIPIO
0,Región Eje Cafetero - Antioquia,5,Antioquia,5.001,Medellín
1,Región Eje Cafetero - Antioquia,5,Antioquia,5.002,Abejorral
2,Región Eje Cafetero - Antioquia,5,Antioquia,5.004,Abriaquí
3,Región Eje Cafetero - Antioquia,5,Antioquia,5.021,Alejandría
4,Región Eje Cafetero - Antioquia,5,Antioquia,5.030,Amagá


## 4.1 Persons of Interests

In [36]:
POI.keys()

dict_keys(['Evaristo Porras', 'Luis Agustín Caicedo Velandia', 'Víctor Manuel Mejía Múnera', 'Miguel Ángel Mejía Múnera', 'Ramón Quintero', 'Pablo Escobar', 'Gonzalo Rodríguez Gacha', 'Diego Fernando Murillo', 'Gilberto Rodríguez Orejuela', 'Miguel Rodríguez Orejuela', 'Leonidas Vargas', 'Luis Enrique Calle Serna', 'Wilber Varela', 'Alberto Orlandez Gamboa', 'Hermágoras González', 'Nicolás Rodríguez Bautista', 'Iván Márquez', 'Gentil Duarte', 'Jorge Gnecco Cerchar', 'Jesús María Aguirre', 'Megateo', 'Vicente Castaño', 'Emiliano Alcides Osorio', 'Edgar Bustamante', 'Manuel Marulanda Vélez', 'Jorge Briceño Suarez Mono', 'Dairo Antonio Úsuga David', 'Salvatore Mancuso', 'Rodrigo Tovar', 'Carlos Castaño Gil', 'Justo Pastor Perafán'])

In [43]:
def replace_accents(w):
    return w.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')

def extract_POI(text):
    if text != text:
        return []
    entities  = []
    temp_text = replace_accents(text.lower())
    for key, value in POI.items():
        temp_key =   replace_accents(key.lower())        
        if temp_key in temp_text: 
                entities += [key] * temp_text.count(temp_key)
        
        if value != "":
            temp_value = replace_accents(value.lower())
            if temp_value in temp_text:
                entities += [key] * temp_text.count(temp_value)
                
    doc = nlp(text)
    for token in doc.ents:
          if (token.label_ == "PER" and token.label_ not in entities):
            entities.append(token.text)
    entities = [e.translate(str.maketrans('', '', punct)) for e in entities]
    return entities

test = df.loc[25]["Text"]
test

'Carrera 9 # 16 -21 Bogotá D.C. PBX: (57) (1 ) 3147300 Línea Nacional: 01 8000 914814 www.defensoria.gov.co Plantilla Vigente desde: 27/06/2017 Bogotá D.C., 5 de marzo de 2018 Doctor GUILLERMO RIVERA FLÓREZ Ministro del Interior Secretaría Técnica de la Comisión Intersectorial para la Respuesta Rápida a las Alertas Tempranas (CIPRAT) Carrera 8 No. 12 B - 31 Ciudad Referencia: ALERTA TEMPRANA DE INMINENCIA N° 0 27-181 debido a la situación de riesgo de vulneracio nes a los DDHH y al DIH de los habitantes del municipio Tarazá – Antioquia, de todos los barrios del casco urbano y de los siguientes corregimientos y veredas: Barro Blanc o, veredas: Tahamí, Doradas Altas, Doradas Bajas, Neri, Tesorito, Potrero Largo, Purí, San Antonio, Tamaco, Barro Blanco; Corregimiento El Doce: Puquí, centro pob lado El Doce; Corregimiento La Caucana, veredas: Santa Clara, La Esperanza, Pecoralia, Agualin da, La Primavera, Rancho Viejo, Pecora, La Caucana, La Cabaña, Piedra Brava, La Cidra, El Porvenir, Bla

In [44]:
ent  = extract_POI(test)

In [45]:
len(ent)

59

In [46]:
df = pd.DataFrame(ent, columns = ['text'])
df['text'].nunique()

49

In [47]:
df['text'].unique()

array(['Doctor GUILLERMO RIVERA FLÓREZ Ministro del Interior Secretaría Técnica',
       'vulneracio nes', 'Doradas Altas', 'Cañón de Iglesias', 'Iglesias',
       'Magdalena Medio', 'Ramiro Vanoy', 'Cuco Vanoy',
       'Bloque Barro Blanco', 'Tomás', 'Paisas', 'Rastrojos',
       'favorecid os', 'Frente Virgilio Peralta', 'José Felipe Reyes',
       'aba', 'Francisco Morales Peñate', 'Rubén Darío', 'impon er',
       'Virgilio Peralta Arenas', 'Son', 'pr', 'Así',
       'Francisco Morelos Peñate', 'Miguel Pérez', 'David Alexis Narváez',
       'señor Alexis', 'Uré', 'Alexa', 'l', 'Yesica Osorio',
       'Intendente Wilson', 'Nelson Ramírez', 'Arley Ortiz Rivas',
       'Ramírez', 'Desplazamiento', 'Restricciones',
       'Bloque Virgilio Peralta Arenas', 'lgunas', 'tonaciones',
       'ASOCBAC', 'q ue', 'nción', 'Jaideza bi', 'Jaidukamá',
       'Dirección Pos', 'Claudia Rojas', 'Rafael Hernando Navarro',
       'Mauricio Redondo Valencia Archivado'], dtype=object)

In [27]:
test

'Ramón Quintero'

In [32]:
df["POI"] = df["Text"].apply(extract_POI)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

# 4.2. Extract Actors

In [ ]:
def extract_actors(text):
    if text != text:
        return []
    
    entities  = []
    temp_text = replace_accents(text.lower())
    
    #grupos
    for key, value in grupos.items():
        temp_key =  replace_accents(key.lower())
        temp_value = replace_accents(value.lower()) if value !="" else ""
        
        if temp_key in temp_text: 
                entities += [key] * temp_text.count(temp_key)
        if temp_value in temp_text:
                entities += [key] * temp_text.count(temp_value)
                
    #narco_grupos           
    for value in narco_grupos:
        if replace_accents(value.lower()) in temp_text:
            entities += [value] * temp_text.count(replace_accents(value.lower()))
                
            
    doc = nlp(text)
    for token in doc.ents:
          if (token.label_ == "ORG" and token.label_ not in entities):
            entities.append(token.text)
    entities = [e.translate(str.maketrans('', '', punct)) for e in entities]
    return entities

In [ ]:
test = df.loc[25]["Text"]
doc = nlp(test)
ents = []
for token in doc.ents:
    ents.append([token.label_, token.text])
df= pd.Dataframe(ents, columns=['label', 'text'])
df.sort_values('label', ascending=True)

In [ ]:
df["Actors"] = df["Text"].apply(extract_actors)

# 5. Saving text in csv

In [ ]:
df.to_csv("data/alertas.csv", sep="|", index=False)

# 6. Export HTML

In [140]:
!jupyter nbconvert --to html 5_Actores_del_Conflicto.ipynb

[NbConvertApp] Converting notebook 4_Distribution_of_documents_Departamentos.ipynb to html
[NbConvertApp] Writing 1281710 bytes to 4_Distribution_of_documents_Departamentos.html
